# Call Center Cleanup

For this assignment, we will be working with call center data. You can start working on the assignment after the first lesson on Exploratory Data Analysis with pandas. Make sure to read the whole assignment before starting anything! As you code along in the Jupyter notebook, you are asked to make note of the results of your analysis. Do so by clicking on the results box and adding your notes beside each question.

## Business Issue and Understanding

You are working for a company that has two call centers: the North Call Center and the South Call Center. The company is looking to possibly hire five additional reps to enhance customer experience. Your task is to explore how efficient the current reps are in each branch to determine which branch would benefit from additional assistance.

### How the Call Center Works

Call center representatives are assigned queues. When calls are assigned to a queue, the call is assigned to the next person in line in the queue. After a call is assigned to a representative, the amount of time between assignment and the call starting is divided into busy minutes and not ready minutes. If the call is incoming and a customer is waiting on the phone for a rep, the time is split into three categories: busy minutes, not ready minutes, and incoming call wait time. Once the rep has the customer on the phone, there might be during call wait time, where the call is put on hold while the rep gets an answer for the customer.

### Notes about the Dataset

If you haven't worked in a call center before, these notes might help you throughout your analysis.

* The call purpose is tagged for each call.
* The time of the call is tagged in 1 hour blocks starting at 9:00 AM and ending at 5:00 PM.
* Calls are tagged as incoming or outgoing.
* Reps are assigned to queues. When the reps are working, they take calls in the order of their queue.
* A call that is dropped due to technical error or missed by the center because they have reached maximum capacity is a lost call.
* An abandoned call is when the customer hangs up because they have been waiting for too long.
* Busy Minutes: the amount of time after a call comes in or needs to go out where the assigned rep is not available because they are busy with other customers.
* Not Ready Minutes: the amount of time after a call comes in or needs to go out where the assigned rep is not available because they are not ready (for example, getting water).
* Incoming Wait Time - amount of time after assigned rep is available to take the call customer waits for representative to pick up a call. This is tracked in seconds.
* During Call Wait Time - amount of time during call that customer has to wait for representative

## Getting Started

You have two CSVs at your disposal, `NorthCallCenter.csv` and `SouthCallCenter.csv`. Import the appropriate libraries and create two dataframes, one called `north_df` and one called `south_df`.

In [47]:
# Import libraries with aliases
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create two new dataframes
north_df = pd.read_csv("NorthCallCenter.csv")
south_df = pd.read_csv("SouthCallCenter.csv")

##Standardizing data to avoid issues I ran into later in the process

# Drop unnamed index columns
north_df = north_df.loc[:, ~north_df.columns.str.contains("^Unnamed")]
south_df = south_df.loc[:, ~south_df.columns.str.contains("^Unnamed")]

# Strip whitespace from all string columns
for df in [north_df, south_df]:
    str_cols = df.select_dtypes(include="object").columns
    df[str_cols] = df[str_cols].apply(lambda x: x.str.strip())

# Define numeric columns that must be numeric
numeric_cols = [
    "Busy Minutes",
    "Not Ready Minutes",
    "Incoming Wait Time",
    "During Call Wait Time",
    "Calls",
    "Lost Call",
    "Abandoned"
]

# Convert numeric columns
for df in [north_df, south_df]:
    df[numeric_cols] = df[numeric_cols].apply(
        pd.to_numeric, errors="coerce"
    )

# Replace NaNs in wait time with 0 (no wait)
for df in [north_df, south_df]:
    df["Incoming Wait Time"] = df["Incoming Wait Time"].fillna(0)

# Fill NaNs for count-based columns with 0
count_cols = ["Calls", "Lost Call", "Abandoned", "Incoming Wait Time"]

for df in [north_df, south_df]:
    df[count_cols] = df[count_cols].fillna(0)
    df[count_cols] = df[count_cols].astype(int)


# Final validation
north_df.info()
south_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Branch                 247 non-null    object 
 1   Call Purpose           249 non-null    object 
 2   Time Block             255 non-null    object 
 3   Incoming or Outgoing   248 non-null    object 
 4   Queue                  248 non-null    object 
 5   Rep ID                 255 non-null    object 
 6   Sale                   249 non-null    object 
 7   Lost Call              256 non-null    int64  
 8   Abandoned              256 non-null    int64  
 9   Busy Minutes           250 non-null    float64
 10  Not Ready Minutes      248 non-null    float64
 11  Incoming Wait Time     256 non-null    int64  
 12  During Call Wait Time  248 non-null    float64
 13  Calls                  256 non-null    int64  
dtypes: float64(3), int64(4), object(7)
memory usage: 28.1+ KB


## Task 1: Exploratory Data Analysis

Time to do some EDA! In the process of learning more about the two datasets, answer the following questions. Use the code blocks below to begin cleaning your data. At the end of the section, record your answers.

#### EDA Question 1A:  How many reps are in the North branch?  

In [48]:
# EDA Question 1A solution below:
north_df["Rep ID"].unique().size

11

#### EDA Question 1B:  How many reps are in the South branch?  

In [49]:
# EDA Question 1B solution Below:
south_df["Rep ID"].unique().size

11

#### EDA Question 2A:  What is the average busy minutes, not ready minutes, incoming wait time, and during call wait time for the North branch? 

In [50]:
# EDA Question 2A solution Below:
north_df[["Busy Minutes", "Not Ready Minutes", "Incoming Wait Time", "During Call Wait Time"]].mean()


Busy Minutes             9.988000
Not Ready Minutes        1.915323
Incoming Wait Time       1.035156
During Call Wait Time    2.967742
dtype: float64

#### EDA Question 2B:  What is the average busy minutes, not ready minutes, incoming wait time, and during call wait time for the South branch? 

In [51]:
# EDA Question 2B solution Below:
south_df[["Busy Minutes", "Not Ready Minutes", "Incoming Wait Time", "During Call Wait Time"]].mean()


Busy Minutes             10.054140
Not Ready Minutes         1.910828
Incoming Wait Time        1.203822
During Call Wait Time     3.076433
dtype: float64

#### EDA Question 3:  What is the number of calls taken for each time block(Both North and South combined)?

In [52]:
# EDA Question 3 solution Below:
north_calls = north_df.groupby("Time Block")["Calls"].sum()
south_calls = south_df.groupby("Time Block")["Calls"].sum()

calls_by_timeblock = north_calls + south_calls
calls_by_timeblock




Time Block
10:00 AM       99.0
11:00 AM       56.0
12:00 PM      120.0
1:00 PM        40.0
2:00 PM        65.0
3:00 PM        73.0
4:00 PM        53.0
5:00 PM        43.0
9:00 AM        11.0
Time Block      NaN
Name: Calls, dtype: float64

##### Record your answers for the EDA section below:
- 1a) Reps in the North Branch = 9
- 1b) Reps in the South Branch = 11


- 2a) North Branch, (round to the nearest hundredth):
    Busy Minutes             9.987755
    Not Ready Minutes        1.914286
    Incoming Wait Time       3.048780
    During Call Wait Time    2.967347
- 2b) South Branch, (round to the nearest hundredth):
    Busy Minutes             10.054140
    Not Ready Minutes         1.910828
    Incoming Wait Time        3.000000
    During Call Wait Time     3.076433
- 3) Total Calls taken for all branches per time block:
> - 9:00AM   =  10
> - 10:00AM  =  99
> - 11:00AM  =  56
> - 12:00PM  =  120
> - 1:00PM   =  40  
> - 2:00PM   =  65
> - 3:00PM   =  73
> - 4:00PM   =  53
> - 5:00PM   =  43



## Task 2: Cleaning Your Data 
Now you need to clean up the datasets. When cleaning the datasets, you may find that there isn't dirty data to clean. That is okay! Some questions you need to answer about the data sets.
* Add additional code blocks as needed to show how you came to your conclusions. Add comments in your code blocks to help others understand your thinking. 

In [ ]:
#replacing missing "Incoming Wait Time" values with 0 for calls without wait times.
north_df["Incoming Wait Time"] = north_df["Incoming Wait Time"].fillna(0)
south_df["Incoming Wait Time"] = south_df["Incoming Wait Time"].fillna(0)
#Change from decimal to interger for "Incoming Wait Time"
north_df["Incoming Wait Time"] = north_df["Incoming Wait Time"].astype(int)
south_df["Incoming Wait Time"] = south_df["Incoming Wait Time"].astype(int)
north_df["Incoming Wait Time"].unique() #Added 0 and intergers
#Mean is corrected.



#### Cleaning Question 1:  Is there any missing data and if yes explain what you would do with the missing data and why?

#### Record your answer below:

Answer 1:
One uncollected column to explore would be "Call Resolution", with (YES/NO) data. In this situation we could see how effective the representatives are when resolving a client's issue or giving them a solution. Along with call times, this could be a huge factor in staffing insights and overall organizational procedures.


#### Record your answer below:

Answer 2: Our "Wait Time" data does not include values for calls without a wait time (0)min, excluding them from being counted when we calculate our mean "Wait Time". Fixing this code would allow us to better estimate how long the average guest waits.


#### Cleaning Question 2:  In the North data set, there are two separate "YES" values for a sale. Why and how did you rectify the issue?

In [ ]:
# Cleaning Question 2 solution below:
# Issue involves space before second single quote
north_df["Sale"] = north_df["Sale"].str.strip()
north_df["Sale"].unique() 

##### Record your answer by below:
> Your Answer: To solve this issue I used string.strip to elimate the extra spaces causing the issue.

#### Cleaning Question 3:  Are there duplicates in the two data sets? If there are, how did you handle them?

In [ ]:
# Cleaning Question 3 solution below:

north_df.drop_duplicates(inplace=True)
south_df.drop_duplicates(inplace=True)


##### Record your answer below:
> Your Answer: There are duplicate rows present in both dataframes. I cleaned them with drop duplicates function.

#### Cleaning Question 4:  Is any of the data in the two data sets unnecessary? If yes, how did you handle it?

In [ ]:
#Dropping Unnamed Columns
north_df = north_df.drop(columns=[col for col in north_df.columns if "Unnamed" in col])
south_df = south_df.drop(columns=[col for col in south_df.columns if "Unnamed" in col])

south_df.head()


##### Record your answer below:
Each dataset has a column indicating the call number (Unnamed:), this matches with our index already, making it redundant. I chose to remove these columns using .drop(columns).

## Task 3: Data Manipulation
Before you begin answering any questions, combine the two datasets together to create a third dataframe called df. You can use this third dataframe to compare the two call centers to company-wide trends.
* Add additional code blocks as needed to show how you came to your conclusions. Add comments in your code blocks to help others understand your thinking. Record your answer below.

In [ ]:
# Create dataframe for the entire company named df
combined_df = pd.concat([north_df, south_df], ignore_index=True)
combined_df.sample(50)


#### Manipulation Question 1:  Group by Rep ID and sum the resulting structure. Sort by calls to determine which rep in each branch has the highest number of calls.

In [ ]:
# Manipulation Question 1 solution below:
combined_df = combined_df[combined_df["Branch"] != "Branch"]
combined_df["Calls"] = pd.to_numeric(combined_df["Calls"], errors="coerce")


top_rep_per_branch = (combined_df
        .groupby(["Branch", "Rep ID"])["Calls"].sum()
        .reset_index()
        .sort_values(["Branch", "Calls"],ascending=[True, False])
        .groupby("Branch").head(1))

top_rep_per_branch


##### Record your answer below
Rep with the hightest number of calls and their total calls:
- North Branch = Brent with 37 calls
- South Branch = Eric with 35 calls

#### Manipulation Question 2:  The average call abandonment rate is a KPI when it comes to determining call center efficiency. As you may recall, abandoned calls are calls where the customer hangs up due to long call times. What is the average call abandonment rate for each branch and the whole company? Do any of these fall out of the optimal range of 2-5%?

In [ ]:
# Manipulation Question 2 solution below:
abandonment_rate_all =combined_df["Abandoned"].mean()
# Rate = 0.019677
# Results in optimal KPI of less than 2% abandonded calls.
Branch_abandonment_rate = (combined_df
        .groupby(["Branch"])["Abandoned"].mean())
Branch_abandonment_rate
# Branch
# North    0.028571
# South    0.012739
           # Abandoned
#North Branch is within range of 2% to 5% abandoned calls.
#South Branch exceeding KPI range of 2% to 5% abandoned calls.

##### Record your answer below:
Average Call Abandonment Rates (round to the nearest hundredth):
- North Branch = 0.028571
- South Branch = 0.012739
- Company Wide = 0.019677
- Do any of these fall out of the optimal range of 2-5%? No, All results show KPI is well within the optimal zone, and in some cases outperforming expectations. 

#### Manipulation Question 3:  Service level is another KPI when it comes to measuring call center efficiency. Service level is the percentage of calls answered within a specific number of seconds. In the case of your employer, their ideal time frame is 2 seconds. What is the percentage of calls answered within 2 seconds for each branch and the entire company?

In [ ]:
# Manipulation Question 3 solution below:
combined_df["Within_2_Seconds"] = combined_df["Incoming Wait Time"] <= 2
service_level_by_branch = (
    combined_df
    .groupby("Branch")["Within_2_Seconds"]
    .mean()
    * 100)

service_level_by_branch

company_service_level = combined_df["Within_2_Seconds"].mean() * 100
company_service_level



##### Record your answer below:
Percentage of calls answered within 2 seconds, include # of calls:
- North Branch = 80.000000
- South Branch = 75.159236
- Company Wide = 77.280857

#### Manipulation Question 4: For each branch and the entire company, what is the average speed of answer?

In [ ]:
# Manipulation Question 4 solution below:
asa_by_branch = (
    combined_df
    .groupby("Branch")["Incoming Wait Time"]
    .mean())
asa_by_branch

company_asa = combined_df["Incoming Wait Time"].mean()
company_asa


##### Record your answer by below:
Average speed of answer (ASA) (rounded to nearest hundredth):
- North Branch in seconds = (ASA) is 1.02 seconds
- South Branch in seconds = (ASA) is 1.20 seconds
- Company Wide in seconds = (ASA) is 1.12 seconds

## Task 4: Visualization

Create a visualization for each of the following questions. Some of the code to handle aggregating and storing data may be written for you. For each visualization, you choose the chart style that you feel suits the situation best. Make note of the chart style you chose and why. 

*NOTE Some questions you may decide to use more than one chart and or chart style.

#### Visualization 1:  What is the average abandonment rate per queue?

In [ ]:
# Calculate total abandoned calls and total calls per queue
queue_totals = combined_df.groupby("Queue")[["Abandoned", "Calls"]].sum()

# Calculate abandonment rate
abandonment_rates = queue_totals["Abandoned"] / queue_totals["Calls"]

abandonment_rates
# Queue
# A    0.041667
# B    0.020134
# C    0.007246
# D    0.017045

# Plot the abandonment rates
plt.figure(figsize=(10, 6))
abandonment_rates.plot(kind="bar")
plt.title("Abandonment Rates by Queue")
plt.xlabel("Queue")
plt.ylabel("Abandonment Rate")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Create visualization 1 here
# Import any additional libraries needed with alias

# The dictionary abandonment_rates has the data you need.
abandonment_rates = {}
queues = ["A", "B", "C", "D"]
queue_dict = combined_df.groupby("Queue").sum()

for q in queues:
    abandonment_rates[q] = queue_dict.loc[q, "Abandoned"] / queue_dict.loc[q, "Calls"]

abandonment_rates
#Your code below:


##### Record your answer below:

> Chart style you chose and why: 

#### Visualization 2: What is the service level percentage and average speed of answer for each rep in the North Branch?

In [ ]:
north_plt = (
    north_df
    .groupby("Rep ID")["Incoming Wait Time"]
    .mean()
    .reset_index()
    .rename(columns={"Incoming Wait Time": "ASA"})
)
# Incoming calls only
incoming_calls = north_df[north_df["Incoming or Outgoing"] == "Incoming"]

# Incoming calls answered within 2 seconds
quick_calls = incoming_calls[incoming_calls["Incoming Wait Time"] <= 2]

# Counts per rep
quick_stats = quick_calls.groupby("Rep ID")["Calls"].sum().reset_index()
total_stats = incoming_calls.groupby("Rep ID")["Calls"].sum().reset_index()

# Merge and calculate %
service_level = pd.merge(quick_stats, total_stats, on="Rep ID", suffixes=("_quick", "_total"))
service_level["Service Level %"] = (
    service_level["Calls_quick"] / service_level["Calls_total"] * 100
)

service_level = service_level[["Rep ID", "Service Level %"]]

north_rep_metrics = pd.merge(north_plt, service_level, on="Rep ID")
north_rep_metrics

plt.figure(figsize=(10,6))
plt.bar(north_rep_metrics["Rep ID"], north_rep_metrics["ASA"])
plt.title("Average Speed of Answer by Rep (North Branch)")
plt.xlabel("Rep ID")
plt.ylabel("Seconds")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(10,6))
plt.bar(north_rep_metrics["Rep ID"], north_rep_metrics["Service Level %"])
plt.title("Service Level % by Rep (North Branch)")
plt.xlabel("Rep ID")
plt.ylabel("Service Level (%)")
plt.xticks(rotation=45)
plt.show()


##### Record your answer below:

> Chart style you chose and why: 

#### Visualization 3: For each type of call purpose, how many calls are outgoing vs. incoming?

In [ ]:
# Create visualization 3 here:
# The three dictionaries, complaints, sales_support, and product_support, have the information you need

purpose_group = df.groupby("Call Purpose")
call_purpose = ["Complaint", "Product Support", "Sales Support"]
purpose_counts = purpose_group["Incoming or Outgoing"].value_counts()
print(purpose_counts)

complaints = purpose_counts["Complaint"].to_dict()
sales_support = purpose_counts["Sales Support"].to_dict()
product_support = purpose_counts["Product Support"].to_dict()

#Your Code Here:


##### Record your answer below:

> Chart style you chose and why: 

## Task 5: Summarize Your Work

With what you know now about the two call centers and the entire company, answer the following questions. Note that while this is subjective, you should include relevant data to back up your opinion.


#### Summary Question 1:  Using KPIs such as average abandonment rate, service level and average speed of answer, in your opinion, which one of the two branches is operating more efficiently? Why?

##### Record your answer below:
> Your Answer: 



#### Summary Question 2: Based on the number of reps in each branch and how quickly the reps are working, in your opinion, which branch would benefit from the extra help?

##### Record your answer below:
> Your Answer: 


#### Summary Question 3: Now that you have explored the datasets, is there any data or information that you wish you had in this analysis?

##### Record your answer below:
> Your Answer: 


## Bonus Mission
Create a visualization that answers this question: For each call purpose, how many calls (incoming and outgoing) take place in each time block?
##### Record your answer below:

> Chart style you chose and why: 

In [ ]:
# Create your Bonus Mission visualization here!
call_times = df[["Time Block", "Call Purpose", "Incoming or Outgoing", "Calls"]]

# Use groupby to plot based on time blocks:

# Use groupby and get_group to select which call purpose to plot: